In [1]:
%load_ext autoreload
%autoreload 1

import logging
import pickle
import sqlite3
import sys

import numpy as np
import pandas as pd

sys.path.append('..')
import loteca.data.interim.matches as matches
%aimport loteca.data.interim.matches

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

In [2]:
# load loteca
loteca = pd.read_pickle('../data/pre/lotecas_matches.pkl')
loteca = loteca[loteca.roundno >= 366]  # exclude old rounds

# load betexplorer
conn = sqlite3.connect('../data/raw/betexplorer.sqlite3')
betexp = pd.read_sql_query('SELECT id, league_category, date, teamH, teamA, score, scoremod FROM matches', conn)
conn.close()

betexp.date = pd.to_datetime(betexp.date, dayfirst=True)
betexp.score = betexp.score.str.strip()
betexp['goalsH'] = [int(score.split(':')[0]) if score else np.nan for score in betexp.score]
betexp['goalsA'] = [int(score.split(':')[1]) if score else np.nan for score in betexp.score]

# load dict
with open('../data/interim/teams_ltb.pkl', mode='rb') as fp:
    teamsd = pickle.load(fp)

In [3]:
loteca0 = loteca[~loteca.happened]  # TODO

In [4]:
# the ones we are sure

loteca1 = loteca[loteca.happened]
results1 = matches.generate_matches_dict(loteca1, betexp, teamsd, logger=logger,
                                             
                                         flexible_date=True,
                                         min_team_rigid_points=2, 
                                         min_team_flex_points=2,
                                         ignore_score=False, 
                                         ignore_score_if_mod=False,
                                         return_teams=False)

len(results1)

INFO:root:creating dataframe cache
INFO:root:on match 1/5856
INFO:root:on match 501/5856
INFO:root:on match 1001/5856
INFO:root:on match 1501/5856
INFO:root:on match 2001/5856
INFO:root:on match 2501/5856
INFO:root:on match 3001/5856
INFO:root:on match 3501/5856
INFO:root:on match 4001/5856
INFO:root:on match 4501/5856
INFO:root:on match 5001/5856
INFO:root:on match 5501/5856


3764

In [32]:
# discover new teams

loteca2 = loteca1[~loteca1.index.isin(results1)]

results2, newteams2 = matches.generate_matches_dict(loteca2, betexp, teamsd, logger=logger,                                         
                                         flexible_date=True,
                                         min_team_rigid_points=1, 
                                         min_team_flex_points=2,
                                         ignore_score=False, 
                                         ignore_score_if_mod=False,
                                         return_teams=True)
    
print(len(results2), len(newteams2))

INFO:root:creating dataframe cache
INFO:root:on match 1/2092
INFO:root:on match 501/2092
INFO:root:on match 1001/2092
INFO:root:on match 1501/2092
INFO:root:on match 2001/2092


1006 0


In [37]:
# update dict

teamsd3 = teamsd.copy()
for k, v in newteams2.items():
    teamsd3[k] |= v 


# discover new teams (more flexble)

loteca3 = loteca2[~loteca2.index.isin(results2)]

results3, teams3 = matches.generate_matches_dict(loteca3, betexp, teamsd3, logger=logger,                                         
                                         flexible_date=True,
                                         min_team_rigid_points=1,
                                         min_team_flex_points=1,
                                         ignore_score=False, 
                                         ignore_score_if_mod=False,
                                         return_teams=True)
    
print(len(results3), len(teams3))

INFO:root:creating dataframe cache
INFO:root:on match 1/1086
INFO:root:on match 501/1086
INFO:root:on match 1001/1086


82 18


In [43]:
betexp[betexp.id.isin(results3.values())][betexp.teamA == 'Guarany']
loteca.loc[7665]

# http://www.betexplorer.com/soccer/brazil/campeonato-cearense-2013/crato-guarany/zcDegFVh/
# http://www.betexplorer.com/soccer/brazil/campeonato-cearense-2013/crato-horizonte/4hgz5CU4/

/home/andre/anaconda3/envs/loteca/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,id,league_category,date,teamH,teamA,score,scoremod,goalsH,goalsA
49723,zcDegFVh,brazil,2013-02-03,Crato,Guarany,2:2,,2.0,2.0


In [46]:
loteca[loteca.roundno == 548]

# look at the dates here...
# it's not right

# lookin in loteca, everything is in march!

# oh man, the timestamps match
# that means they used the raw date string to generate the timestamp

# and they did it mixing the day and month...

,roundno,gameno,date,teamH,goalsH,teamA,goalsA,happened
7658,548,1,2013-02-03,FLAMENGO/RJ,0,BOTAFOGO/RJ,2,True
7659,548,2,2013-02-02,VASCO DA GAMA/RJ,3,FLUMINENSE/RJ,2,True
7660,548,3,2013-02-03,PARANÁ/PR,2,PARANAVAÍ/PR,1,True
7661,548,4,2013-02-02,BRAGANTINO/SP,1,SOROCABA/SP,2,True
7662,548,5,2013-02-03,METROPOLITANO/SC,2,CRICIÚMA/SC,1,True
7663,548,6,2013-03-03,GUARANI/SP,0,MOGI MIRIM/SP,2,True
7664,548,7,2013-02-03,SANTA CRUZ/PE,0,SALGUEIRO/PE,1,True
7665,548,8,2013-02-02,CRATO/CE,2,HORIZONTE/CE,2,True
7666,548,9,2013-02-03,CENTRAL/PE,0,SPORT/PE,1,True
7667,548,10,2013-02-02,SÃO MATEUS/ES,4,VITÓRIA/ES,1,True


In [38]:
teams3

# 'HORIZONTE/CE': {'Guarany'},
# 'IPATINGA/MG': {'Betim'},
# 'ITUIUTABA/MG': {'Boa'},

defaultdict(set,
            {'ATHLETIC BILBAO/ESP': {'Ath Bilbao'},
             'ATLÉTICO/BA': {'Alagoinhas'},
             'BOCA JÚNIOR/SE': {'Boca Junior'},
             'CAMBURIÚ/SC': {'Camboriu'},
             'COLÔNIA/DEU': {'FC Koln'},
             'DEPORTI LA CORUNA/ESP': {'Dep. La Coruna'},
             'DEPORTIVO LA CORUÑA/ESP': {'Dep. La Coruna'},
             'ESPORTIVO/RS': {'Bento Goncalves'},
             'GRÊMIO PRUDENTE/SP': {'Barueri'},
             'HERTHA BERLIM/DEU': {'Hertha Berlin'},
             'HORIZONTE/CE': {'Guarany'},
             'IPATINGA/MG': {'Betim'},
             'ITUIUTABA/MG': {'Boa'},
             'MOTO CLUBE/MA': {'Moto Club'},
             'NEW CASTLE/ING': {'Newcastle'},
             'OLYMPIQUE LYONNAIS/FRA': {'Lyon'},
             'VILA NOVA/MG': {'Villa Nova MG'},
             'WEST BROMWICH/ING': {'West Brom'}})

In [31]:
# discover new teams

loteca4 = loteca3[~loteca3.index.isin(results3)]

results4, newteams4 = matches.generate_matches_dict(loteca4, betexp, teamsd3, logger=logger,                                         
                                         flexible_date=True,
                                         min_team_rigid_points=1, 
                                         min_team_flex_points=2,
                                         ignore_score=False, 
                                         ignore_score_if_mod=False,
                                         return_teams=True)
    
print(len(results4), len(newteams4))

INFO:root:creating dataframe cache
INFO:root:on match 1/1086
INFO:root:on match 501/1086
INFO:root:on match 1001/1086


0 0


In [21]:
newteams2

defaultdict(set,
            {'AMERICANO/RJ': {'Americano FC'},
             'BOA ESPORTE CLUBE/MG': {'Boa'},
             'BOA ESPORTE/MG': {'Boa'},
             'BOLOGNA FC/ITA': {'Bologna'},
             'BORUSSIA DORTMUND/DEU': {'Dortmund'},
             'BRAGANTINO/SP': {'Bragantino'},
             'BRASIL/RS': {'Brasil de Pelotas'},
             'CAGLIARI CALCIO/ITA': {'Cagliari'},
             'CAXIAS/RS': {'SER Caxias'},
             'CELTA DE VIGO/ESP': {'Celta Vigo'},
             'CENTRAL/PE': {'Central SC'},
             'COLO COLO/BA': {'Colo C.'},
             'CUIABÁ/MT': {'Cuiaba Esporte'},
             'ESTANCIANO/SE': {'Estanciano EC'},
             'FC BARCELONA/ESP': {'Barcelona'},
             'FERNANDÓPOLIS/SP SUB 20': {'Fernandopolis U20'},
             'FLUMINENSE/BA': {'Fluminense de Feira'},
             'FORMOSA/GO': {'Bosque Formosa'},
             'GRÊMIO BARUERI/SP': {'Barueri'},
             'GUARANI/CE': {'Guarani de Juazeiro'},
             'GUARANI/MG'

In [12]:
loteca[loteca.index.isin(results2.keys())]

,roundno,gameno,date,teamH,goalsH,teamA,goalsA,happened
5114,366,5,2009-06-06,PONTE PRETA/SP,5,PORTUGUESA DESPORTOS/SP,2,True
5116,366,7,2009-06-06,VASCO DA GAMA/RJ,0,SÃO CAETANO/SP,0,True
5122,366,13,2009-06-07,SPORT/PE,4,FLAMENGO/RJ,2,True
5127,367,4,2009-06-13,GUARANI/SP,0,VASCO DA GAMA/RJ,0,True
5130,367,7,2009-06-13,A B C/RN,1,VILA NOVA/GO,0,True
5133,367,10,2009-06-13,SPORT/PE,0,ATLÉTICO/PR,1,True
5142,368,5,2009-06-20,VILA NOVA/GO,2,ATLÉTICO/GO,3,True
5145,368,8,2009-06-20,SANTO ANDRÉ/SP,2,SPORT/PE,1,True
5154,369,3,2009-06-27,FIGUEIRENSE/SC,1,VASCO DA GAMA/RJ,1,True
5164,369,13,2009-06-28,SPORT/PE,3,GRÊMIO/RS,1,True


In [ ]:
for k, v in results6.items():
    display(loteca.loc[k])
    display(betexp[betexp.id == v])

In [ ]:
from IPython.display import display

id1 = 5188
id2 = ['ADnFZrWr', 'QDf0HHdP']
display(loteca.loc[id1])
display(betexp[betexp.id.isin(id2)])

# Algorithm

# Some failed BetExplorer names

Not all BetExplorer team names are right. Here are some of them that were captured in this algorithm.

In [ ]:
# http://www.betexplorer.com/soccer/brazil/serie-b-2009/america-rn-betim/zuc6o4ea/
# it's Ipatinga, not Betim
betexp.loc[22800]

In [ ]:
# http://www.betexplorer.com/soccer/brazil/campeonato-paulista-2010/santo-andre-barueri/d4IyAGi2/
# It's Grêmio Prudente, not Barueri
betexp.loc[18934]

In [ ]:
# http://www.betexplorer.com/soccer/brazil/serie-b-2011/barueri-portuguesa/4rHNNkEH/
# This one is right, but it contradicts with the example above
betexp.loc[32436]